# Widgets

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageCleaner

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

## Images

### DatasetFormatter
The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate
1,0.181563,0.110704,0.036310
2,0.098868,0.080926,0.024534


In [ ]:
learn.save('stage-1')

We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)

In [ ]:
data = (ImageItemList.from_folder(path)
                     .no_split()
                     .label_from_folder()
                     .databunch())

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system. To use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) we must first use `DatasetFormatter().from_toplosses` to get the suggested indices for misclassified images.

### ImageCleaner

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

RuntimeError: invalid argument 2: k not in range for dimension at /opt/conda/conda-bld/pytorch_1544176307774/work/aten/src/TH/generic/THTensorMoreMath.cpp:1185

In [ ]:
ImageCleaner(ds, idxs, path)

You can also use ImageCleaner to find duplicates in your dataset. For this, you need to run `DatasetFormatter().from_similars` and then `ImageCleaner` with `duplicates=True`.

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn, layer_ls=[0,7,1], pool_dim=1)

Getting activations...


Computing similarities...


In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())